# PROJECT_VIVINO

In [ ]:
# initializing settings to run vivino_analysis_combination.ipynb

## import libraries
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.orm import Session, sessionmaker, registry

## setting verbosity level of sqlalchemy
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.ERROR)

## open vivino.db from data directory
vivino_engine = create_engine('sqlite:///../data/vivino.db', echo=True)

## establish and instantiate session as vivino_session
vivino_session = sessionmaker(bind=vivino_engine)
vivino_session = vivino_session()

## create a registry for the vivino_session
mapper_registry = registry()
Base = mapper_registry.generate_base()


In [ ]:
# Create an inspector
inspector = inspect(vivino_engine)

# Get the table names from the inspector
tablenames_list = inspector.get_table_names()

# query schema for all tables in vivino.db
for table in tablenames_list:
    query_schema = vivino_engine.execute(
        f"PRAGMA table_info({table});"
        )
    print(f"\n{table} schema:")
    for row in query_schema:
        print(row)


## QUESTION 1

We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
- Those with good ratings but not a big amount of ratings
- Natural wines?

#### CATEGORY 1 - BEST RETAIL WINE

In [ ]:
# Identifying 5 best retail wines 
query_wines_natural = vivino_engine.execute(
    """
    SELECT 
        wines.id AS wine_id,
        wines.name AS wine_name,
        wines.is_natural AS is_natural,
        wines.ratings_average,
        wines.ratings_count,
        regions.name AS region_name,
        countries.name AS country_name
    FROM 
        wines
    LEFT JOIN 
        regions ON wines.region_id = regions.id
    LEFT JOIN 
        countries ON regions.country_code = countries.code
    WHERE 
        wines.ratings_average >= 4.5
        AND wines.ratings_count >= 650
        AND wines.is_natural = 1
    ORDER BY
        ratings_count DESC;
    """
)

# fetch query_wines_natural
fetch_query_wines_natural = query_wines_natural.fetchall()

# initialize fetch_query_wines_natural into pandas dataframe named df_wines_top5_natural
df_wines_top5_natural = pd.DataFrame(
    fetch_query_wines_natural,
    columns=query_wines_natural.keys()
)

# display df_wines_top5_natural
display(df_wines_top5_natural)


We can recommend the wines listed above because those wines are natural, European Wines substantiated with a solid customer reviews and rating.

#### CATEGORY 2 - BEST VINTAGE WINE

In [ ]:
# Identifyoing 5 best vintage wines
query_vintages= vivino_engine.execute(
    """
    SELECT 
        avg(rank) AS avg_rank,
        vintages.name as vintage_wine_name,
        vintages.wine_id AS vintage_wine_id,
        vintages.ratings_average AS ratings_average,
        vintages.ratings_count AS ratings_count,
        vintages.price_euros AS retail_price_euros,
        vintages.bottle_volume_ml AS bottle_volume_ml,
        vintage_toplists_rankings.rank AS rank,
        vintage_toplists_rankings.previous_rank AS previous_rank,
        regions.name AS region_name,
        countries.name AS country_name
    FROM 
        vintages
    LEFT JOIN 
        wines ON vintages.wine_id = wines.id
    LEFT JOIN 
        regions ON wines.region_id = regions.id
    LEFT JOIN 
        countries ON regions.country_code = countries.code
    INNER JOIN 
        vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
    WHERE 
        vintages.ratings_count >= 3000
        AND vintages.ratings_average >= 4.5
        AND vintage_toplists_rankings.rank = vintage_toplists_rankings.previous_rank
        AND (vintage_toplists_rankings.rank <= 10 OR vintage_toplists_rankings.previous_rank <= 10)
    GROUP BY 
        vintages.wine_id 
    ORDER BY 
        avg_rank ASC;
    """
)

# initialize best_vintages_df
fetch_query_vintages = query_vintages.fetchall()

best_vintages_df = pd.DataFrame(
    fetch_query_vintages,
    columns=query_vintages.keys()
)

# drop 'price_discounted_from', 'price_discount_percentage', 'id', 'name',

# display best_vintages_df
display(best_vintages_df)



We recommend these vintage wines due to the fact that they have been consistently marking their place in our vintages_toplists_rankings table throughout multiple years, and their positions are substantiated with a significant amount of ratings and overall average rating score. 

## QUESTION 2
We have a marketing budget for this year. Which country should we prioritise and why?
- Those with high user count?
- those we low user count?
- Those with a upcoming user popularity

In [ ]:
# CRITERIA 1 - COUNTRIES WITH HIGH USER COUNTS

## query countries table to yield top 5 countries based on user counts

query_top_5_user_count = vivino_engine.execute(
    """
    SELECT 
        countries.name AS country_name,
        countries.users_count AS user_count
    FROM 
        countries
    ORDER BY 
        user_count DESC
    LIMIT 5;
    """
)

## initialize top_5_user_count_df
fetch_query_top_5_user_count = query_top_5_user_count.fetchall()

top_5_user_count_df = pd.DataFrame(
    fetch_query_top_5_user_count,
    columns=query_top_5_user_count.keys()
)

## display top_5_user_count_df
display(top_5_user_count_df)

In [ ]:
# CRITERIA 3 - COUNTRIES WITH UPCOMING USER POPULARITY

## initializing query to yield upcoming popular countries
query_upcoming_popular_country = vivino_engine.execute(
    """
    SELECT 
        countries.name AS country_name,
        SUM(wines.ratings_count) AS ratings_count
    FROM 
        wines
    LEFT JOIN 
        regions ON wines.region_id = regions.id
    LEFT JOIN 
        countries ON regions.country_code = countries.code
    GROUP BY 
        country_name
    ORDER BY 
        ratings_count DESC;
"""
)

## initialize upcoming_popular_country_df
fetch_query_upcoming_popular_country = query_upcoming_popular_country.fetchall()

upcoming_popular_country_df = pd.DataFrame(
    fetch_query_upcoming_popular_country,
    columns=query_upcoming_popular_country.keys()
)

## display upcoming_popular_country_df
display(upcoming_popular_country_df)


Based on the information above, we suggest to focus the marketing resources on establishing customer base in these following countries 
 1. Italy (listed as Italie)
 2. France (listed as France)
 3. Spain (listed as Espagne)
 4. United States of America (listed as États-Unis)
 5. Germany (listed as Allemagne)

 The first 3 countries are countries from which the wines are highly sought after by customers in general, based on the customer engagement toward the wines the respective countries, which is measured through ratings count. On top of that, they are also countries with a substantial amount of customers as well. 
 The last 2 countries are countries from which the wines are gaining more popularity. However, they are countries with a significant amount of customers who uses vivino. 
 

## QUESTION 3

We have detected that a big cluster of customers like a specific combination of tastes.
We have identified a few primary keywords that match this.
We would like you to find all the wines that have those keywords. 
To ensure the accuracy of our selection, ensure that more than 10 users confirmed those keywords.
 Also, identify the group_name related to those keywords. 
 - coffee
- toast
- green apple
- cream
- citrus

In [ ]:
# Identifying the associated primary flavor_keyword per wine, as well as the flavor_group it belongs to.
query_keyword_flavor_profile = vivino_engine.execute(
    """
    SELECT 
        wines.id AS wine_id,
        wines.name AS wine_name,
        keywords.name AS flavor_keyword,
        flavor_groups.name AS flavor_group,
        keywords_wine.keyword_type AS keyword_type,
        regions.name AS region_name,
        countries.name AS country_name,
        wines.ratings_average AS ratings_average,
        wines.ratings_count AS ratings_count
    FROM 
        wines
    LEFT JOIN 
        keywords_wine ON wines.id = keywords_wine.wine_id
    LEFT JOIN 
        keywords ON keywords_wine.keyword_id = keywords.id
    LEFT JOIN 
        flavor_groups ON keywords_wine.group_name = flavor_groups.name
    LEFT JOIN 
        regions ON wines.region_id = regions.id
    LEFT JOIN 
        countries ON regions.country_code = countries.code
    WHERE 
        keywords_wine.count >= 10
        AND keyword_type = 'primary'
    ORDER BY 
        ratings_count DESC;
    """
)

## initialize keyword_flavor_profile_df
fetch_query_keyword_flavor_profile = query_keyword_flavor_profile.fetchall()

keyword_flavor_profile_df = pd.DataFrame(
    fetch_query_keyword_flavor_profile,
    columns=query_keyword_flavor_profile.keys()
)

## group flavor_keyword per flavor_group
keyword_flavor_profile_df = keyword_flavor_profile_df.groupby(
    [
        'wine_id', 
        'wine_name', 
        'flavor_group', 
        'keyword_type', 
        'region_name', 
        'country_name', 
        'ratings_average', 
        'ratings_count'
        ])['flavor_keyword'].apply(', '.join).reset_index()


## display keyword_flavor_profile_df
display(keyword_flavor_profile_df)



In [ ]:
# Identifying the group_name of the requested flavor keywords
query_requested_keywords = vivino_engine.execute(
    """
    SELECT 
        count(wines.name) as keyword_match_counter, 
        keywords_wine.wine_id,
        wines.name AS wine_name,
        keywords_wine.keyword_type AS keyword_type,
        keywords_wine.group_name AS flavor_group
    FROM 
        keywords_wine 
    JOIN 
        wines on wine_id = wines.id 
    WHERE 
        keywords_wine.count > 10 
        AND keyword_id IN (
            SELECT id 
            FROM keywords 
            WHERE name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
        )
        AND keyword_type = 'primary' 
        GROUP BY wine_id 
        ORDER BY keyword_match_counter DESC
        LIMIT 20;
    """
)

## INitialize keyword_flavor_profile_df
fetch_query_requested_keywords = query_requested_keywords.fetchall()

requested_keyword_df = pd.DataFrame(
    fetch_query_requested_keywords,
    columns=query_requested_keywords.keys()
)

## display keyword_flavor_profile_df
display(requested_keyword_df)


 - 'keyword_flavor_profile_df' highlights the 'flavor_group' of each associated 'flavor_key' per 'wine_name'. 

 - 'requested_keyword_df' highlights the 'flavor_group' name in which the requested 'flavor_key' belong to. 
     - 'keyword_match_counter' column acts as an indicator that suggests the matched 'wine_name' fulfills all of the requested 'flavor_key'
     - The requested 'flavor_key' are: 'coffee', 'toast', 'green apple', 'cream', 'citrus'
     - The requested 'flavor_key' belongs to 'citrus_fruit' 'flavor_group'


## QUESTION 4
We would like to select wines that are easy to find all over the world. 
1. Find the top 3 most common grape all over the world 
2. For each grape, give us the the 5 best rated wines.

In [ ]:
# 1. Find the top 3 most common grape all over the world 

query_4 = vivino_engine.execute(
    """
    SELECT
        grapes.name AS grape_name,
        count(most_used_grapes_per_country.country_code) AS num_available_countries,
        most_used_grapes_per_country.wines_count
    FROM
        most_used_grapes_per_country
    LEFT JOIN 
        grapes ON most_used_grapes_per_country.grape_id = grapes.id
    LEFT JOIN 
        countries ON most_used_grapes_per_country.country_code = countries.code
    WHERE
        most_used_grapes_per_country.wines_count > 100000
    GROUP BY 
        most_used_grapes_per_country.grape_id
    ORDER BY
        num_available_countries DESC
    LIMIT 3;
    """
)

## Fetch query_4 and initialize into df_most_common_grapes
fetch_query_4 = query_4.fetchall()
df_most_common_grapes = pd.DataFrame(
    fetch_query_4,
    columns=query_4.keys()
)

## display df_most_common_grapes
display(df_most_common_grapes)



Based on the information above, here are the top 3 grape varieties based on their usage in different wines throughout the world
 1. Cabernet Sauvignon
 2. Merlot
 3. Chardonnay

In [ ]:
# Churning 5 best rated wines per df_most_common_grapes['grape_name']
top_grapes = df_most_common_grapes['grape_name'].tolist()

## query top 5 cabernet sauvignon wines
query_cabernet_sauvignon = vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN 
        regions ON wines.region_id = regions.id
    JOIN 
        countries ON regions.country_code = countries.code
    JOIN 
        most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN 
        grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[0]}'
        AND wines.ratings_average >= 4.5
    ORDER BY 
        review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_cabernet_sauvignon and initialize into df_cabernet_sauvignon
fetch_query_cabernet_sauvignon = query_cabernet_sauvignon.fetchall()
df_cabernet_sauvignon = pd.DataFrame(
    fetch_query_cabernet_sauvignon,
    columns=query_cabernet_sauvignon.keys()
)

## display df_cabernet_sauvignon
display(df_cabernet_sauvignon)

## query top 5 merlot wines
query_merlot =  vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN 
        regions ON wines.region_id = regions.id
    JOIN 
        countries ON regions.country_code = countries.code
    JOIN 
        most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN 
        grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[1]}'
        AND wines.ratings_average >= 4.5
    ORDER BY 
        review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_merlot and initialize into df_merlot
fetch_query_merlot = query_merlot.fetchall()
df_merlot = pd.DataFrame(
    fetch_query_merlot,
    columns=query_merlot.keys()
)

## display df_merlot
display(df_merlot)

## query top 5 Chardonnay wines
query_chardonnay =  vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN 
        regions ON wines.region_id = regions.id
    JOIN 
        countries ON regions.country_code = countries.code
    JOIN 
        most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN 
        grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[2]}'
        AND wines.ratings_average >= 4.5
    ORDER BY 
        review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_chardonnay and initialize into df_chardonnay
fetch_query_chardonnay = query_chardonnay.fetchall()
df_chardonnay = pd.DataFrame(
    fetch_query_chardonnay,
    columns=query_chardonnay.keys()
)

## display df_chardonnay
display(df_chardonnay)



The information above highlights 5 best-rated wines for each grape variety based on their overall 'ranks' and 'review_amount'

For those who likes wines using Cabernet Sauvignon grape variety, we are happy to recommend these wines
 1. Cabernet Sauvignon - Napa Valley - USA
 2. Opus One - Napa Valley - USA
 3. Valbuena 5º - Ribera del Duero - Spain
 4. Ribera del Duero - Ribera del Duero - Spain
 5. Almaviva - Puente Alto - Chili

## QUESTION 5

We would to give create a country leaderboard,
give us a visual that shows the average wine rating for each country

LEADERBOARD CATEGORY
 - Countries with the highest count of top-rated wines (young wines, vintage wines)



In [ ]:
# Identify countries with highest count of top - rated young wines
query_retail_wines = vivino_engine.execute(
    """
    SELECT
        countries.name AS country_name,
        count(wines.name) AS wine_count,
        avg(wines.ratings_average) AS avg_rating,
        sum(wines.ratings_count) AS total_reviews
    FROM
        wines
    INNER JOIN 
        regions ON wines.region_id = regions.id
    INNER JOIN 
        countries ON regions.country_code = countries.code
    GROUP BY 
        countries.name
    HAVING
        total_reviews >= 100000
    ORDER BY 
        wine_count DESC;
    """
)

# initialize retail_wines_df
fetch_query_retail_wines = query_retail_wines.fetchall()
retail_wines_df = pd.DataFrame(
    fetch_query_retail_wines,
    columns=query_retail_wines.keys()
)

# display retail_wines_df
display(retail_wines_df)


In [ ]:
# Identify countries with highest count of top - rated vintage wines
query_country_vintages = vivino_engine.execute(
    """
SELECT
    countries.name AS country_name, 
    CAST(ROUND(avg(rank) + 0.5) AS INTEGER) AS avg_rank,
    count(vintages.name) as vintage_count,
    avg(vintages.ratings_average) AS avg_rating,
    sum(vintages.ratings_count) AS total_reviews,
    avg(vintages.price_euros) AS retail_price_euros    
FROM vintages
LEFT JOIN wines ON vintages.wine_id = wines.id
LEFT JOIN regions ON wines.region_id = regions.id
LEFT JOIN countries ON regions.country_code = countries.code
INNER JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE vintage_toplists_rankings.rank = vintage_toplists_rankings.previous_rank
GROUP BY countries.name
ORDER BY vintage_count DESC;
"""
)

# initialize country_vintages_df
fetch_query_country_vintages = query_country_vintages.fetchall()
country_vintages_df = pd.DataFrame(
    fetch_query_country_vintages,
    columns=query_country_vintages.keys()
)

# unlimited display country_vintages_df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(country_vintages_df)


THE AWARD GOES TO
- ITALY
- FRANCE
- SPAIN

### EXTRA ANALYSIS

In [ ]:
# expandings wines table to include flavor_keywords and flavor_groups

query_wines_expanded = vivino_engine.execute(
    """
    SELECT 
        wines.id AS wine_id,
        wines.name AS wine_name,
        wines.is_natural AS is_natural,
        wines.ratings_average,
        wines.ratings_count,
        regions.name AS region_name,
        countries.name AS country_name,
        GROUP_CONCAT(DISTINCT keywords.name) AS flavor_keywords,
        GROUP_CONCAT(DISTINCT flavor_groups.name || ': ' || keywords.name) AS flavor_group_keywords
    FROM 
        wines
    LEFT JOIN 
        regions ON wines.region_id = regions.id
    LEFT JOIN 
        countries ON regions.country_code = countries.code
    LEFT JOIN
        keywords_wine ON wines.id = keywords_wine.wine_id
    LEFT JOIN
        keywords ON keywords_wine.keyword_id = keywords.id
    LEFT JOIN
        flavor_groups ON keywords_wine.group_name = flavor_groups.name
    GROUP BY
        wines.id, wines.name, wines.is_natural, wines.ratings_average,
        wines.ratings_count, regions.name, countries.name
    ORDER BY
        ratings_count DESC;
    """
)

# fetch query_wines_expanded
fetch_query_wines_expanded = query_wines_expanded.fetchall()

# initialize fetch_query_wines_expanded into pandas dataframe named df_wines_expanded
df_wines_expanded = pd.DataFrame(
    fetch_query_wines_expanded,
    columns=query_wines_expanded.keys()
)

# unlimited display df_wines_expanded
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_wines_expanded.head(10))

In [ ]:
# close connection and session	
vivino_session.close()
vivino_engine.dispose()
